In [ ]:
import pandas as pd

import src

In [ ]:
ches = pd.read_csv(src.PATH / "data/CHES/CHES2019V3.csv")

ches = ches.loc[(ches.country == 3), ["party", "people_vs_elite", "antielite_salience"]]

ches["party"] = ches["party"].replace({"GRUNEN": "Grüne", "LINKE": "DIE LINKE."})

ches = ches.rename({"antielite_salience": "elite", "people_vs_elite": "pplcentr"}, axis=1)

seats_cdu = 200
seats_csu = 45

row = pd.Series({"party": "CDU/CSU"})
for var in ["pplcentr", "elite"]:
    row[var] = (
        (ches.loc[ches.party == "CSU", var].values[0] * seats_csu)
        + ((ches.loc[ches.party == "CDU", var].values[0] * seats_cdu))
    ) / (seats_cdu + seats_csu)

ches = pd.concat([ches.T, row], axis=1).T

ches["pplcentr"] = ches["pplcentr"].astype(float)
ches["elite"] = ches["elite"].astype(float)

In [ ]:
ches

,party,pplcentr,elite
21,CDU,1.700000,0.750000
22,SPD,2.450000,1.350000
23,FDP,2.350000,1.250000
24,Grüne,4.900000,1.800000
25,DIE LINKE.,4.350000,4.450000
26,CSU,2.842105,1.550000
27,AfD,8.315789,9.700000
28,Piraten,8.900000,8.800000
29,DieTier,9.666667,10.000000
0,CDU/CSU,1.909774,0.896939


In [ ]:
df_ours = pd.read_csv(src.PATH / "data/figure_1_numbers.csv")
df_ours = df_ours[df_ours.electoral_term == 19]
df_ours["variable"] = df_ours["variable"].replace(
    {"(a) Anti-Elitism": "elite", "(b) People-Centrism": "pplcentr"}
)
df_ours = df_ours[df_ours.variable.isin(["elite", "pplcentr"])]
df_ours = df_ours.pivot(index="Party", columns="variable", values="value").reset_index()

In [ ]:
df = pd.merge(
    ches, df_ours, how="inner", left_on="party", right_on="Party", suffixes=["_ches", ""]
).drop("Party", axis=1)

In [ ]:
df.corr(method="pearson", numeric_only=True)

,pplcentr_ches,elite_ches,elite,pplcentr
pplcentr_ches,1.000000,0.926460,0.927202,0.746326
elite_ches,0.926460,1.000000,0.895517,0.767615
elite,0.927202,0.895517,1.000000,0.774918
pplcentr,0.746326,0.767615,0.774918,1.000000
